# March Madness Prediction

## Overview

### Goal
Submissions are based on the Brier Score, the goal will be to minimize the brier score between the predicted probabilities and the actual game outcomes. The Brier score measures the accuracy of probablistic predition, in this case the mean square error. 

The brier score can be thought of as a cost function that measures the average squared difference between the predicted probabilities and the actual outcomes.

$$
Brier = \frac{1}{N} \sum_{i=1}^{N} (p_i - o_i)^2
$$

where $p_i$ is the predicted probability of the event and $o_i$ is the actual outcome. The Brier score can span across all items in a set of N predictions.

Therefore, minimizing the Brier score will result in a more accurate prediction.



## Import Libraries
Numpy for numerical operations
Pandas for data manipulation
Matplotlib, Seaborn, Plotly for plotting



In [2]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import xgboost as xgb
import sklearn as sk


## Load Data

We want to get a baseline model in which we can improve upon. In order to do this effectively, I will use a class structure to store all the data and functions that will be used along the process. This will make it easier to improve and maintain changes to the prediction process.


In [7]:
class MarchMadnessPredictor:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.data = None
        self.teams = None
        self.seeds  = None
        self.submission = None
        self.all_compact_results = None
        self.all_detailed_results = None
        self.tourney_compact_results = None
        self.tourney_detailed_results = None
        self.model = None
        self.calibration_model = None

    def load_data(self):
        
        """
        Set up a data dictionary that will store the data for each file. e.g.
        self.data = {
            'teams': [DataFrame with teams data],
            'games': [DataFrame with games data],
            'players': [DataFrame with players data]
        }
        """

        files = glob.glob(self.data_dir + '*.csv')
        self.data = {file.split('\\')[-1].split('.')[0]: pd.read_csv(file, encoding='latin-1') for file in files}

        self.submission = self.data['SampleSubmissionStage1']

        teams = pd.concat([self.data['MTeams'], self.data['WTeams']])
        teams_spelling = pd.concat([self.data['MTeamSpellings'], self.data['WTeamSpellings']])
        teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
        teams_spelling.columns = ['TeamID', 'TeamNameCount']
        self.teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
        #print(self.teams.head())

        season_compact_results = pd.concat([self.data['MRegularSeasonCompactResults'], self.data['WRegularSeasonCompactResults']]).assign(ST='S')
        season_detailed_results = pd.concat([self.data['MRegularSeasonDetailedResults'], self.data['WRegularSeasonDetailedResults']]).assign(ST='S')
        tourney_compact_results = pd.concat([self.data['MNCAATourneyCompactResults'], self.data['WNCAATourneyCompactResults']]).assign(ST='T')
        tourney_detailed_results = pd.concat([self.data['MNCAATourneyDetailedResults'], self.data['WNCAATourneyDetailedResults']]).assign(ST='T')

        # Extract numeric seed value from seed string
        seeds = pd.concat([self.data['MNCAATourneySeeds'], self.data['WNCAATourneySeeds']])
        seeds['SeedValue'] = seeds['Seed'].str.extract('(\d+)').astype(int)
        self.seeds = seeds
        print(self.seeds)

        """
        Load the game data with additional derived features.
        Combines regualr season and tournament results
        """

        # Combine all game results
        all_compact_results = pd.concat([season_compact_results, tourney_compact_results])
        all_detailed_results = pd.concat([season_detailed_results, tourney_detailed_results])

        # Add derived features to compact results
        all_compact_results['ScoreDiff'] = all_compact_results['WScore'] - all_compact_results['LScore']
        all_compact_results['HomeAdvantage'] = all_compact_results['WLoc'].map({'H': 1, 'N': 0, 'A': -1})
        
        # Add derived features to detaifled results
        all_detailed_results['ScoreDiff'] = all_detailed_results['WScore'] - all_detailed_results['LScore']
        all_detailed_results['HomeAdvantage'] = all_detailed_results['WLoc'].map({'H': 1, 'N': 0, 'A': -1})

         # Calculate shooting percentages (handling division by zero)
        all_detailed_results['WFGPct'] = np.where(all_detailed_results['WFGA'] > 0, 
                                                all_detailed_results['WFGM'] / all_detailed_results['WFGA'], 0)
        all_detailed_results['WFG3Pct'] = np.where(all_detailed_results['WFGA3'] > 0, 
                                                all_detailed_results['WFGM3'] / all_detailed_results['WFGA3'], 0)
        all_detailed_results['WFTPct'] = np.where(all_detailed_results['WFTA'] > 0, 
                                                all_detailed_results['WFTM'] / all_detailed_results['WFTA'], 0)
        all_detailed_results['LFGPct'] = np.where(all_detailed_results['LFGA'] > 0, 
                                                all_detailed_results['LFGM'] / all_detailed_results['LFGA'], 0)
        all_detailed_results['LFG3Pct'] = np.where(all_detailed_results['LFGA3'] > 0, 
                                                all_detailed_results['LFGM3'] / all_detailed_results['LFGA3'], 0)
        all_detailed_results['LFTPct'] = np.where(all_detailed_results['LFTA'] > 0, 
                                                all_detailed_results['LFTM'] / all_detailed_results['LFTA'], 0)
        
        # Add statistical differences
        all_detailed_results['ReboundDiff'] = (all_detailed_results['WOR'] + all_detailed_results['WDR']) - \
                                            (all_detailed_results['LOR'] + all_detailed_results['LDR'])
        all_detailed_results['AssistDiff'] = all_detailed_results['WAst'] - all_detailed_results['LAst']
        all_detailed_results['TurnoverDiff'] = all_detailed_results['WTO'] - all_detailed_results['LTO']
        all_detailed_results['StealDiff'] = all_detailed_results['WStl'] - all_detailed_results['LStl']
        all_detailed_results['BlockDiff'] = all_detailed_results['WBlk'] - all_detailed_results['LBlk']
        all_detailed_results['FoulDiff'] = all_detailed_results['WPF'] - all_detailed_results['LPF']

        # Add seed information to tournament games
        tourney_compact = all_compact_results[all_compact_results['ST'] == 'T'].copy()
        tourney_detailed = all_detailed_results[all_detailed_results['ST'] == 'T'].copy()

        # Add winner seeds
        tourney_compact = pd.merge(
            tourney_compact,
            seeds[['Season', 'TeamID', 'SeedValue']],
            how='left',
            left_on=['Season', 'WTeamID'],
            right_on=['Season', 'TeamID']
        )
        tourney_compact.rename(columns={'SeedValue': 'WSeedValue'}, inplace=True)
        tourney_compact.drop('TeamID', axis=1, inplace=True)
        
        tourney_detailed = pd.merge(
            tourney_detailed,
            seeds[['Season', 'TeamID', 'SeedValue']],
            how='left',
            left_on=['Season', 'WTeamID'],
            right_on=['Season', 'TeamID']
        )
        tourney_detailed.rename(columns={'SeedValue': 'WSeedValue'}, inplace=True)
        tourney_detailed.drop('TeamID', axis=1, inplace=True)

        # Add loser seeds
        tourney_compact = pd.merge(
            tourney_compact,
            seeds[['Season', 'TeamID', 'SeedValue']],
            how='left',
            left_on=['Season', 'LTeamID'],
            right_on=['Season', 'TeamID']
        )
        tourney_compact.rename(columns={'SeedValue': 'LSeedValue'}, inplace=True)
        tourney_compact.drop('TeamID', axis=1, inplace=True)
        
        tourney_detailed = pd.merge(
            tourney_detailed,
            seeds[['Season', 'TeamID', 'SeedValue']],
            how='left',
            left_on=['Season', 'LTeamID'],
            right_on=['Season', 'TeamID']
        )
        tourney_detailed.rename(columns={'SeedValue': 'LSeedValue'}, inplace=True)
        tourney_detailed.drop('TeamID', axis=1, inplace=True)

         # Calculate seed difference (lower is better in seeding, so LSeed - WSeed is positive if favorite won)
        tourney_compact['SeedDiff'] = tourney_compact['LSeedValue'] - tourney_compact['WSeedValue']
        tourney_detailed['SeedDiff'] = tourney_detailed['LSeedValue'] - tourney_detailed['WSeedValue']

        # Store all processed data
        self.all_compact_results = all_compact_results
        self.all_detailed_results = all_detailed_results
        self.tourney_compact_results = tourney_compact
        self.tourney_detailed_results = tourney_detailed

        print("All Compact Resullts: \n", self.all_compact_results.head())
        print(self.all_detailed_results.head())
        print(self.tourney_compact_results.head())
        print(self.tourney_detailed_results.head())

    def create_model(self):
        """
        Creates XGBoost models for prediction and calibration.
        """
        # Main prediction model
        self.model = xgb.XGBRegressor(
            n_estimators=500,         # Number of boosting rounds
            learning_rate=0.05,       # Smaller learning rate for better generalization
            max_depth=6,              # Control model complexity
            min_child_weight=3,       # Helps prevent overfitting
            subsample=0.8,            # Use 80% of data for each tree
            colsample_bytree=0.8,     # Use 80% of features for each tree
            objective='reg:squarederror',  # Optimizes for MSE which aligns with Brier score
            random_state=42,
            n_jobs=-1                 # Use all CPU cores
        )
        
        # Calibration model to fine-tune probabilities
        self.calibration_model = xgb.XGBRegressor(
            n_estimators=200,
            learning_rate=0.03,
            max_depth=4,
            min_child_weight=2,
            subsample=0.7,
            colsample_bytree=0.7,
            objective='reg:squarederror',
            random_state=42,
            n_jobs=-1
        )
        
    def display_win_distribution(self, min_season=None, max_season=None, title=None):
        """
        Displays a heatmap of win distribution by seed matchup.
        
        Parameters:
        -----------
        min_season : int, optional
            Minimum season to include in the analysis
        max_season : int, optional
            Maximum season to include in the analysis
        title : str, optional
            Custom title for the plot
        
        Returns:
        --------
        None, displays the plot
        """
        # Filter tournament data by season if specified
        df = self.tourney_compact_results.copy()
        if min_season is not None:
            df = df[df['Season'] >= min_season]
        if max_season is not None:
            df = df[df['Season'] <= max_season]
        
        # Create a crosstab (matrix) of winner seed vs loser seed
        win_matrix = pd.crosstab(
            index=df['LSeedValue'],  # Y-axis: losing seeds
            columns=df['WSeedValue'],  # X-axis: winning seeds
            values=df['Season'],
            aggfunc='count'
        ).fillna(0)
        
        # Ensure all seeds from 1-16 are represented
        all_seeds = list(range(1, 17))
        for seed in all_seeds:
            if seed not in win_matrix.index:
                win_matrix.loc[seed] = 0
            if seed not in win_matrix.columns:
                win_matrix[seed] = 0
        
        # Sort the indices to ensure they're in order from 1-16
        win_matrix = win_matrix.reindex(index=all_seeds, columns=all_seeds)
        
        # Create the heatmap
        fig = sp.make_subplots(
            rows=1, cols=1,
            subplot_titles=[title or f"March Madness Win Distribution by Seed ({min_season or 'All'}-{max_season or 'Present'})"]
        )
        
        # Add the heatmap
        heatmap = go.Heatmap(
            z=win_matrix.values,
            x=win_matrix.columns,
            y=win_matrix.index,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Count'),
            text=[[f"{int(val)} wins" if val > 0 else "" for val in row] for row in win_matrix.values],
            hoverinfo='text',
            hoverongaps=False
        )
        
        fig.add_trace(heatmap)
        
        # Update layout
        fig.update_layout(
            height=600,
            width=700,
            xaxis=dict(
                title='Winning Seed',
                tickmode='linear',
                tick0=1,
                dtick=1
            ),
            yaxis=dict(
                title='Losing Seed',
                tickmode='linear',
                tick0=1,
                dtick=1,
                autorange='reversed'  # Reverse y-axis to have 1 at the top
            )
        )
        
        # Add a diagonal line to indicate "expected" outcomes (lower seed beats higher seed)
        for i in range(1, 17):
            for j in range(1, 17):
                if i == j:
                    continue
                if i < j:  # "Expected" outcome: lower seed (i) beats higher seed (j)
                    # No need to highlight as this is expected
                    pass
                else:  # "Upset": higher seed (i) loses to lower seed (j)
                    if win_matrix.loc[i, j] > 0:
                        # Add a rectangle around upset cells
                        fig.add_shape(
                            type="rect",
                            x0=j-0.5, x1=j+0.5,
                            y0=i-0.5, y1=i+0.5,
                            line=dict(color="red", width=1),
                            fillcolor="rgba(0,0,0,0)"
                        )
        
        # Show the plot
        fig.show()
        
        # Calculate and display some statistics
        total_games = win_matrix.sum().sum()
        expected_wins = sum(win_matrix.iloc[i-1:, i].sum() for i in range(1, 17))  # Sum of lower seeds beating higher seeds
        upset_wins = total_games - expected_wins
        
        print(f"Total Tournament Games: {int(total_games)}")
        print(f"Expected Outcomes (lower seed beats higher seed): {int(expected_wins)} ({expected_wins/total_games:.1%})")
        print(f"Upset Outcomes (higher seed beats lower seed): {int(upset_wins)} ({upset_wins/total_games:.1%})")
        
        # Show the most common upsets
        upset_matrix = win_matrix.copy()
        for i in range(1, 17):
            for j in range(1, 17):
                if i <= j:  # Not an upset
                    upset_matrix.iloc[i-1, j-1] = 0
        
        if upset_matrix.sum().sum() > 0:
            top_upsets = []
            for i in range(1, 17):
                for j in range(1, 17):
                    if i > j and upset_matrix.iloc[i-1, j-1] > 0:
                        top_upsets.append((j, i, int(upset_matrix.iloc[i-1, j-1])))
            
            top_upsets.sort(key=lambda x: x[2], reverse=True)
            
            print("\nTop 5 Most Common Upsets:")
            for w_seed, l_seed, count in top_upsets[:5]:
                print(f"Seed #{w_seed} beating Seed #{l_seed}: {count} times")
                
        


<>:45: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Cardo\AppData\Local\Temp\ipykernel_14520\3731092701.py:45: SyntaxWarning: invalid escape sequence '\d'
  seeds['SeedValue'] = seeds['Seed'].str.extract('(\d+)').astype(int)


In [9]:
if __name__ == '__main__':
    data_dir = 'data/'
    predictor = MarchMadnessPredictor(data_dir)
    predictor.load_data()
    predictor.display_win_distribution()

      Season  Seed  TeamID  SeedValue
0       1985   W01    1207          1
1       1985   W02    1210          2
2       1985   W03    1228          3
3       1985   W04    1260          4
4       1985   W05    1374          5
...      ...   ...     ...        ...
1671    2024  Z12b    3435         12
1672    2024   Z13    3267         13
1673    2024   Z14    3238         14
1674    2024   Z15    3263         15
1675    2024   Z16    3394         16

[4234 rows x 4 columns]
All Compact Resullts: 
    Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT ST  ScoreDiff  \
0    1985      20     1228      81     1328      64    N      0  S         17   
1    1985      25     1106      77     1354      70    H      0  S          7   
2    1985      25     1112      63     1223      56    H      0  S          7   
3    1985      25     1165      70     1432      54    H      0  S         16   
4    1985      25     1192      86     1447      74    H      0  S         12   

   HomeA

C:\Users\Cardo\AppData\Local\Temp\ipykernel_14520\3731092701.py:45: SyntaxWarning:

invalid escape sequence '\d'



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed